In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.gridspec import GridSpec

#データをload
f=np.load('')
#air data h = after air g = before air
h=np.load('')
g=np.load('')

class Fit_trans_refl:
    def __init__(self,i_angle,incpol,thickness):
        self.i_angle = i_angle
        self.incpol = incpol
        self.thickness = thickness
        
    def Fit_Func(self, n, losstan, d, freq_in, angle_i, incpol):
        c = 2.99792458e8
        num=len(d) #; the number of layer not including two ends
        const = np.sqrt((8.85418782e-12)/(4.*np.pi*1e-7)) #SI unit sqrt(dielectric const/permiability)
        # ;-----------------------------------------------------------------------------------
        # ; angle of refraction
        angle = np.zeros(num+2)          # ; angle[0]=incident angle
        angle[0] = angle_i
        for i in range(0,num+1): angle[i+1] = np.arcsin(np.sin(angle[i])*n[i]/n[i+1])
        # ;-----------------------------------------------------------------------------------
        # ; define the frequency span
        l = len(freq_in)
        output = np.zeros((3,l),'complex') # output = dcomplexarr(3,l)
        # ;-----------------------------------------------------------------------------------
        # ; define the effective thickness of each layer
        h = np.zeros(num,'complex')
        n_comparr = np.zeros(len(n),'complex')
        n_comparr[0] = complex(n[0], -0.5*n[0]*losstan[0])
        n_comparr[num+1] = complex(n[num+1], -0.5*n[num+1]*losstan[num+1])
        # ;-----------------------------------------------------------------------------------
        # ; for loop for various thickness of air gap between each layer
        for j in range(0,l):
            for i in range(0,num): 
                n_comparr[i+1] = complex(n[i+1], -0.5*n[i+1]*losstan[i+1])
                h[i] = n_comparr[i+1]*d[i]*np.cos(angle[i+1]) # ;effective thickness of 1st layer

            freq = freq_in[j]
            k = 2.*np.pi*freq/c

            # ;===========================================
            # ; Y: Y[0]=vacuum, Y[1]=1st layer..., Y[num+1]=end side
            Y = np.zeros(num+2,'complex')
            for i in range(0,num+2):
                if (incpol == 1):
                    Y[i] = const*n_comparr[i]*np.cos(angle[i])
                    cc = 1.
                if (incpol == -1):
                    Y[i] = const*n_comparr[i]/np.cos(angle[i])
                    cc = np.cos(angle[num+1])/np.cos(angle[0])

            # ;===========================================
            # ; define matrix for single layer
            m = np.identity((2),'complex')    # ; net matrix
            me = np.zeros((2,2),'complex') # ; me[0]=1st layer, ...
            for i in range(0,num):
                me[0,0] = complex(np.cos(k*h[i]), 0.)
                me[1,0] = complex(0., np.sin(k*h[i])/Y[i+1])
                me[0,1] = complex(0., np.sin(k*h[i])*Y[i+1])
                me[1,1] = complex(np.cos(k*h[i]), 0.)
                m = np.dot(m,me)

            r = (Y[0]*m[0,0]*cc+Y[0]*Y[num+1]*m[1,0]-m[0,1]*cc-Y[num+1]*m[1,1]) / (Y[0]*m[0,0]*cc+Y[0]*Y[num+1]*m[1,0]+m[0,1]*cc+Y[num+1]*m[1,1])
            t = 2.*Y[0] / (Y[0]*m[0,0]*cc+Y[0]*Y[num+1]*m[1,0]+m[0,1]*cc+Y[num+1]*m[1,1])

            output[0,j] = freq+0.j #; unit of [Hz]
            output[1,j] = r
            output[2,j] = t

        return output
    
    
    def Transmittance_FitIndex_1layer_incloss(self, freq, par0, par1): #[Hz]
        if par0 < 1: return 1e30
        if par1 < 0: return 1e30
    
        incpol = self.incpol
        d = np.array([self.thickness])
        angle_i = np.radians(self.i_angle)
        
        n = np.array([1.,par0,1.])
        losstan = np.array([0.,par1,0.])
        
        
        RT = self.Fit_Func( n, losstan, d, freq, angle_i, incpol)
        return np.abs(RT[2])**2
    
    
    def Reflectance_FitIndex_1layer_incloss(self, freq, par0, par1): #[Hz]
        if par0 < 1: return 1e30
        if par1 < 0: return 1e30
    
        incpol = self.incpol
        d = np.array([self.thickness])
        angle_i = np.radians(self.i_angle)

        n = np.array([1.,par0,1.])
        losstan = np.array([0.,par1,0.])
        
        RT = self.Fit_Func( n, losstan, d, freq, angle_i, incpol)
        return np.abs(RT[1])**2
    
#    par_alumina = np.array([5.,5.029e-3,3.06,1e-4])
# ======= fit paramter =======
incpol=-1 # (-1: p-polarization, 1: s-polarization)
inc_angle = 7 # incident angle [deg]
thickness = 1.572e-3 # thickness [m]
n_init = 1.5 # Refractive index
los_init = 4.e-4 # Loss tangent
# =================================


# set fitting parameters
par_index=[n_init, los_init]

# call class
lib_func = Fit_trans_refl(inc_angle,incpol,thickness)

#transmittanceのグラフを作成
f.files
freq_data= f['freq']*1e9
trans_data = 10**(f['data']/10)
#plt.plot(freq_data*1e-9,trans_data)
#plt.grid()
#plt.ylim(0,1.2)

#sigma 生成
hfreq = h['freq']
hdata = 10**(h['data']/10)

gfreq = g['freq']
gdata = 10**(g['data']/10)

#sigma_y = 1-gdata

#sigma_ave = np.sum(np.abs(sigma_y))/1001
#print(sigma_ave)

#airの差　グラフ
#plt.plot(h['freq'],-sigma_y)
#plt.ylabel('after - before')
#plt.xlabel('Frequency[GHz]')
#plt.rcParams["font.size"] = 15
#plt.ylim(-0.006,0.006)
#plt.grid()
#plt.title('0 degrees')


#curvefit 変数 (n, losstan)

poptT, pcovT = curve_fit(lib_func.Transmittance_FitIndex_1layer_incloss, 
                       freq_data, 
                       trans_data, 
                       p0=par_index, 
                       #sigma=[0.0047]*len(freq_data),
                       #sigma = sigma_y,
                       absolute_sigma=False,
                       maxfev=100000,
                       bounds=([1.05, 1e-6], [3.2, 1e-1]))

print('proptT=',poptT)

perr = np.sqrt(np.diag(pcovT))
print('perr=',perr)

fit = lib_func.Transmittance_FitIndex_1layer_incloss(freq_data,poptT[0],poptT[1])

# fitのグラフを作成
#plt.plot(freq_data*1e-9,fit)

d = trans_data - fit

fig = plt.figure(figsize = (8,6))
# サブプロットを8:2で分割
ax1 = fig.add_axes((0, 0.2, 1, 0.4))
ax2 = fig.add_axes((0, 0., 1, 0.2), sharex=ax1)

# 散布図のx軸のラベルとヒストグラムのy軸のラベルを非表示
ax1.tick_params(labelbottom="off")
ax2.tick_params(labelleft="off")

ax1.plot(freq_data*1e-9,fit,'-',zorder = 1)
ax1.plot(freq_data*1e-9,trans_data,'.',zorder =0 )

ax2.plot(freq_data*1e-9,d)

fs = 17
ls = 17


ax1.set_xlabel('Frequency [GHz]',fontsize = fs)
ax1.set_ylabel('Transmittance',fontsize =fs)
ax1.set_ylim(0.2,1.1)
ax1.grid()
#ax1.legend()
ax1.set_title('abs')
ax1.tick_params(labelsize = ls)

ax2.set_xlabel('Frequency [GHz]',fontsize = fs)
ax2.set_ylabel('Data - fit',fontsize =fs)
ax2.set_ylim(-0.07,0.07)
ax2.grid()
#ax2.legend()
ax2.tick_params(labelsize = ls)

#fig.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

def RMS(rms_data):
    rms = np.sum(rms_data**2)/float(len(rms_data))
    rms = np.sqrt(rms)
    return rms
diff = trans_data - fit
print('RMS = ',RMS(diff))